## Testeando API Ollama

Puedes utilizar un modelo de código abierto que se ejecuta localmente a través de Ollama en lugar de OpenAI.

Podrás utilizar esta técnica si prefieres no utilizar APIs de pago.

**Ventajas
1. No hay cargos por API - de código abierto
2. Los datos no salen de tu ordenador

**Desventajas
1. 1. Mucho menos potente que el modelo Frontier

## Recapitulación sobre la instalación de Ollama

Basta con visitar [ollama.com](https://ollama.com) e instalar.

Una vez completado, el servidor ollama ya debería estar funcionando localmente.  
Si visita  
[http://localhost:11434/](http://localhost:11434/)

Deberías ver el mensaje `Ollama se está ejecutando`.  

Si no es así, abre un nuevo Terminal (Mac) o Powershell (Windows) e introduce `ollama serve`.  
A continuación, inténtalo de nuevo con [http://localhost:11434/](http://localhost:11434/).


In [23]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [131]:
!ollama pull deepseek-r1

"ollama" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [105]:
# Constantes

MODEL = "llama3.5"

In [107]:
# Una clase para representar una página web

class Website:
    """
    Una clase para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        headers = {"accept-language": "en-US,en;q=0.9","accept-encoding": "gzip, deflate, br","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36","accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}
        response = requests.get(url,headers=headers)
        print(response.status_code)
        soup = BeautifulSoup(response.text, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [109]:
# Vamos a probar


web = Website("https://esesa.eu")
print(web.title)
print(web.text)

200
ESESA Escuela Superior de Estudios de Empresa - ESESA
Masters
Máster en Marketing y Digital Business
Máster Executive en Gestión de Proyectos y Metodologías Ágiles
Máster Executive en Asesoría Fiscal
Máster en Big Data e Ingeniería de Datos
Máster Executive en Dirección Financiera
Cursos
Experiencia Alumni
Acto de Clausura
Equipo Docente
Eventos
¿Por qué ESESA?
Blog
ESESA
Masters
Máster en Marketing y Digital Business
Máster Executive en Gestión de Proyectos y Metodologías Ágiles
Máster Executive en Asesoría Fiscal
Máster en Big Data e Ingeniería de Datos
Máster Executive en Dirección Financiera
Cursos
Experiencia Alumni
Acto de Clausura
Equipo Docente
Eventos
¿Por qué ESESA?
Blog
© 2025 ESESA
Escuela Superior de Estudios de Empresa
ESESA
, la escuela de negocios de referencia en Málaga desde 1988, te ofrece una formación práctica, innovadora y adaptada a las demandas del mercado actual. Con programas diseñados para profesionales que buscan crecer en sus carreras y recién titulados

## Tipos de indicaciones/prompts

Los modelos como GPT4o han sido entrenados para recibir instrucciones de una manera particular.

Esperan recibir:

**Una indicación del sistema** que les indique qué tarea están realizando y qué tono deben usar

**Una indicación del usuario**: el inicio de la conversación al que deben responder

In [112]:
# Define nuestro mensaje de sistema: puedes experimentar con esto más tarde, cambiando la última oración a "Responder en Markdown en español".

system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

In [114]:
# Una función que escribe un mensaje de usuario que solicita resúmenes de sitios web:

def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt

In [116]:
print(user_prompt_for(web))

Estás viendo un sitio web titulado ESESA Escuela Superior de Estudios de Empresa - ESESA
El contenido de este sitio web es el siguiente;     proporciona un breve resumen de este sitio web en formato Markdown.     Si incluye noticias, productos o anuncios, resúmelos también.

Masters
Máster en Marketing y Digital Business
Máster Executive en Gestión de Proyectos y Metodologías Ágiles
Máster Executive en Asesoría Fiscal
Máster en Big Data e Ingeniería de Datos
Máster Executive en Dirección Financiera
Cursos
Experiencia Alumni
Acto de Clausura
Equipo Docente
Eventos
¿Por qué ESESA?
Blog
ESESA
Masters
Máster en Marketing y Digital Business
Máster Executive en Gestión de Proyectos y Metodologías Ágiles
Máster Executive en Asesoría Fiscal
Máster en Big Data e Ingeniería de Datos
Máster Executive en Dirección Financiera
Cursos
Experiencia Alumni
Acto de Clausura
Equipo Docente
Eventos
¿Por qué ESESA?
Blog
© 2025 ESESA
Escuela Superior de Estudios de Empresa
ESESA
, la escuela de negocios de r

## Mensajes

La API de OpenAI espera recibir mensajes en una estructura particular.
Muchas de las otras API comparten esta estructura:

```
[
    {"role": "system", "content": "el mensaje de sistema va aquí"},
    {"role": "user", "content": "el mensaje de usuario va aquí"}
]

In [119]:
# Puedes ver cómo esta función crea exactamente el formato anterior

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [121]:
messages_for(web)

[{'role': 'system',
  'content': 'Eres un asistente que analiza el contenido de un sitio web y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. Responder en Markdown.'},
 {'role': 'user',
  'content': 'Estás viendo un sitio web titulado ESESA Escuela Superior de Estudios de Empresa - ESESA\nEl contenido de este sitio web es el siguiente;     proporciona un breve resumen de este sitio web en formato Markdown.     Si incluye noticias, productos o anuncios, resúmelos también.\n\nMasters\nMáster en Marketing y Digital Business\nMáster Executive en Gestión de Proyectos y Metodologías Ágiles\nMáster Executive en Asesoría Fiscal\nMáster en Big Data e Ingeniería de Datos\nMáster Executive en Dirección Financiera\nCursos\nExperiencia Alumni\nActo de Clausura\nEquipo Docente\nEventos\n¿Por qué ESESA?\nBlog\nESESA\nMasters\nMáster en Marketing y Digital Business\nMáster Executive en Gestión de Proyectos y Metodologías Ágiles\nMáster Executive en Ase

In [123]:
# Y ahora: llamamos a la función Ollama en lugar de OpenAI.
# Usando ollama en un host diferente
localhost = "localhost"
localhost = "host.docker.internal"

def summarize(url):
    client = ollama.Client(
      host=f'http://{localhost}:11434',
      headers={'x-some-header': 'some-value'}
    )
    website = Website(url)
    messages = messages_for(website)
    response = client.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [125]:
# Y ahora: llamamos a la función Ollama en lugar de OpenAI.
# Usando ollama by default

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [127]:
summarize("https://esesa.eu")

200


ResponseError: model "deepseek-r1" not found, try pulling it first (status code: 404)

In [14]:
# Una función para mostrar esto de forma clara en la salida de Jupyter, usando markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://esesa.eu")

200


Parece que estás leyendo un sitio web de la Escuela Superior de Ingeniería de Sevilla (ESESA) que ofrece diferentes programas de formación para empresas y profesionales. Aquí te presento una versión más corta y resumida del contenido:

**Formación para empresas**

* Formación InCompany: personalizados para tus necesidades
* Reskilling: mejora las habilidades de tu equipo para alcanzar metas estratégicas
* Upskilling: prepara a tu plantilla ante nuevos desafíos
* Onboarding: facilita la integración de nuevas incorporaciones
* Fidelización: motiva a tu personal y aumenta su retención

**Colaboraciones con empresas**

* Nuestro equipo ha trabajado con algunas de las empresas más importantes del sector

**Información básica**

* Edificio Tabacalera, Mód. E0, Avd. Sor Teresa Prat 15, 29003 Málaga
* Teléfono: 952 07 14 51
* Correo electrónico: info@esesa.eu

Espero que esta versión te sea más útil. Si necesitas más información o tienes alguna pregunta específica, no dudes en preguntar.